# 1. 基本的串接方法

在簡單的使用情境中只使用語言模型當然沒問題，但要開發複雜的應用就會需要串接不同的元件 (Components)。這些元件包括語言模型 (LLM)、提示 (Prompt)、輸出解析 (Output Parser)、抽取器 (Retriever)、工具 (Tool)、任何函式 (function)。

最基本的例子就是串接 Prompt、Model、Output Parser 這三種元件 (Components) 

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_setup import ChatOpenAI

# Output parser
response_schemas = [
    ResponseSchema(name="食物", description="推測喜歡的食物種類"),
    ResponseSchema(name="店家", description="有賣上述麵食種類的店家"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Model
model = ChatOpenAI()

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個在新宿的廢棄大樓營業的占卜師兼 Google 地圖的當地嚮導。"),
        ("human", "請告訴我 {person} 喜歡吃什麼種類的食物，並推薦一家餐廳或小吃給他或她。\n\n{format_instructions}\n\n"),
    ],
)
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())

最陽春的方法是自行呼叫所有元件

In [6]:
person = '小明'
model_input = prompt.format_messages(person=person)
model_output = model(model_input)
parser_output = output_parser.parse(model_output.content)
print(parser_output)

{'食物': '日本料理', '店家': '銀座壽司'}


Langchain 提供了更簡潔的方式來串接元件 (componenents)，其名為 Langchain Expression Language (LCEL)。

In [7]:
chain = prompt | model | output_parser

chain.invoke({'person': '小明'})

{'食物': '日本料理', '店家': '新宿餐廳'}

# 2. LCEL 的介紹

## 2.1 Why LCEL

LCEL 其實就是透過 pipeline 的方式組合成一個新的 runnable 物件 (object)。這樣的方法有幾個好處。
1. 容易上手: 不需要記各個元件 (components) 怎麼操作，Langchain 會自動串好
2. 易懂易改: 透過其自創的LCEL，使用者可以更容易了解執行流程中的每個步驟，也就更能自己做改動
3. 模組化設計: 做好的 runnable 還可以跟其他元件 (components) 或 runnable 串接
4. 各式功能: 透過 LCEL 做出來的 runnable 自帶各種介面和功能，包括:
   - 平行化處理 (parallel processing): 將大量複數的輸入 (input samples) 透過有效率的平行化處理一次執行
   - 非同步處理 (Asynchronous): 自帶非同步介面 
   - 串流 (streaming): 一個字一個字的產出
   - Input/Output Schema: 自帶輸入和輸出的型值檢查 (type check)
   - LangServe 支援: 一行程式碼就可以為其建立 UI 和 API
   - LangSmith 支援: 可以監控每一步驟的輸出
   - Langchain Hub 支援: 可以下載別人做好的 Chain，也可以分享自己做得不錯的 Chain


In [8]:
chain.batch([{'person': '小明'}, {'person': '小美'}])

[{'食物': '日本料理', '店家': '新宿御苑附近有一家名為「すし善」的日本料理店，提供新鮮的壽司和其他傳統日式料理。'},
 {'食物': '拉麵', '店家': '一蘭拉麵'}]

## 2.2 基本概念介紹
LCEL 中的物件可區分為兩個種類 Components 和 Runnable。

**Components:**
|                 | Input                                | Output                |
| :-------------- | :----------------------------------- | :-------------------- |
| Prompt          | Dictionary                           | PromptValue           |
| LLM / ChatModel | String / List[Message] / PromptValue | String / Message      |
| OutputParser    | Output of LLM / ChatModel            | Depends on the parser |
| Retriever       | String                               | List[Documents]       |
| Tool            | Depends on the tool                  | Depends on the tool   |

**Runnable:**<br>
底下有不同的子類別，常見的包括 `RunnableSequence`, `RunnableLambda`, `RunnableMap`, `RunnablePassthrough`

## 3. 串接任意函式 (RunnableLambda)

能用 "|" 符號來自動串接的是 Component 和 Runnalbe。

因此要串接任意函式 (Function)，需要用 `RunnableLambda` 來包覆 (wrap) 該函式

In [1]:
from langchain.schema.runnable import RunnableLambda

def dummy_fn_a(argument_a: str) -> str:
    return argument_a + "_dummya"
def dummy_fn_b(argument_b: str) -> str:
    return argument_b + "_dummyb"
def dummy_fn_c(argument_c: str) -> str:
    return argument_c + "_dummyc"

dummy_runnable = RunnableLambda(dummy_fn_a) | RunnableLambda(dummy_fn_b) | RunnableLambda(dummy_fn_c)
dummy_runnable.invoke("白金之星")

'白金之星_dummya_dummyb_dummyc'

但其實只要其中一方是 Runnable 或 Component，另一方的 function 會自動用 RunnableLambda 包起來

In [2]:
RunnableLambda(dummy_fn_a) | dummy_fn_b | dummy_fn_c

RunnableLambda(...)
| RunnableLambda(...)
| RunnableLambda(...)

# 4. 資料傳遞路徑
最基本的資料傳遞路徑是一直線
input -> componentA -> componentB -> output

## 4.1 並聯 (RunnableMap)

分支後再結合
```
     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
      Combine
```

In [1]:
from langchain.schema.runnable import RunnableMap, RunnableLambda, RunnableSequence

def combine(responses: dict):
    print("[RunnableMap 的輸出]:\n")
    print(responses)
    return f"兩種情況\n{responses['branch1_result']}\n{responses['branch2_result']}"

branched_runnable = RunnableMap({
    "branch1_result": lambda situation: situation + "好哇",
    "branch2_result": lambda situation: situation + "因對貓毛過敏而婉拒",
}) | combine 

inputs = "要不要去她家看會後空翻的貓咪。"
output = branched_runnable.invoke(inputs)
print("\n[合體後的輸出]:\n")
print(output)

[RunnableMap 的輸出]:

{'branch1_result': '要不要去她家看會後空翻的貓咪。好哇', 'branch2_result': '要不要去她家看會後空翻的貓咪。因對貓毛過敏而婉拒'}

[合體後的輸出]:

兩種情況
要不要去她家看會後空翻的貓咪。好哇
要不要去她家看會後空翻的貓咪。因對貓毛過敏而婉拒


我們這邊用了 `RunnableMap` 來包覆 `dict`，其型別應為 `dict[str, Runnable | 任何可被自動轉換成 Runnable 的東西]`

`RunnableMap` 的輸入 (input) 會平行地給每一個作為 dict value 的 runnable (在這個例子就是兩個 lambda function)，並且該 runnalbe 吐出的輸出 (output) 會成為 `RunnableMap` 輸出的 dictionary 中的 dict value (見 "[[RunnableMap 的輸出]")。

而跟上面的 function 一樣，只要 "|" 的其中一方是 Runnable，另一方的 dictionary 會自動的被轉換成 `RunnableMap`

In [2]:
branched_runnable = {
    "branch1_result": lambda situation: situation + "好哇", 
    "branch2_result": lambda situation: situation + "因對貓毛過敏而婉拒",
} | RunnableLambda(combine) 
output = branched_runnable.invoke(inputs)
print("\n[合體後的輸出]:\n")
print(output)

[RunnableMap 的輸出]:

{'branch1_result': '要不要去她家看會後空翻的貓咪。好哇', 'branch2_result': '要不要去她家看會後空翻的貓咪。因對貓毛過敏而婉拒'}

[合體後的輸出]:

兩種情況
要不要去她家看會後空翻的貓咪。好哇
要不要去她家看會後空翻的貓咪。因對貓毛過敏而婉拒


## 4.2 執行多條候選路徑中的其中一條

在這個章節的範例中我們會用一些簡單的函式來作為要執行的 runnable 和選擇要執行哪個 runnable 的 runnable，但你可以想像這些 runnables 都是可以用語言模型來改寫的。

In [1]:
from langchain.schema.runnable import RunnableBranch, RunnableLambda

love_question_solver = RunnableLambda(lambda x: x + "\n回答: 分手")
money_question_solver = RunnableLambda(lambda x: x + "\n回答: 躺平")
backup_solver = RunnableLambda(lambda x: x + "\n回答: 我不清楚")

### 4.2.1 Match 式 (RunnableBranch)
如同 Python 的 `match` 一樣 (其他語言可能叫 `switch`)，符合該條件則執行該條件下的程式

In [2]:
match_runnable = RunnableBranch(
    (lambda question: "愛" in question, love_question_solver), # (輸出為 boolean 的 runnable,  執行的 runnable)
    (lambda question: "錢" in question, money_question_solver), # (輸出為 boolean 的 runnable,  執行的 runnable)
    backup_solver,
)
print(match_runnable.invoke("我想知道他是不是還愛我?"), end='\n\n') # love_question_solver
print(match_runnable.invoke("不夠錢買房怎麼辦?"), end='\n\n') # money_question_solver
print(match_runnable.invoke("1+1=?"), end='\n\n') # backup_solver

我想知道他是不是還愛我?
回答: 分手

不夠錢買房怎麼辦?
回答: 躺平

1+1=?
回答: 我不清楚



### 4.2.2 Routing 式 

In [3]:
def route(question: str):
    if "錢" in question:
        return money_question_solver
    elif "愛" in question:
        return love_question_solver
    else:
        return backup_solver

route_runnable = RunnableLambda(route)
print(route_runnable.invoke("我想知道他是不是還愛我?"), end='\n\n') # love_question_solver
print(route_runnable.invoke("不夠錢買房怎麼辦?"), end='\n\n') # money_question_solver
print(route_runnable.invoke("1+1=?"), end='\n\n') # backup_solver

我想知道他是不是還愛我?
回答: 分手

不夠錢買房怎麼辦?
回答: 躺平

1+1=?
回答: 我不清楚



**想想看：**

兩種選擇方式各有什麼優缺點？

<details>
<summary>參考</summary>
Match 適合簡單的條件判斷或有明確的優先順序要求，否則的話 Router 式可以只透過一個 LLM Call 就能判別要走哪個分支。
</details>

# 5. 為傳遞的資料添加新值
一般的情況是
```
資料Ａ --> 資料Ｂ
```
但有時我們想要的是 
```
資料Ａ --> 資料A, 資料Ｂ=處理(資料Ａ)
```

In [1]:
from langchain.prompts import PromptTemplate

inputs = {'e1': 2, 'e2': 3}

expected_outputs = {'e1': 2, 'e2': 3, 'e3': 5}

template = """
數字1: {e1}
數字2: {e2}
數字3: {e3}
下一個數字是什麼"""
prompt = PromptTemplate.from_template(template)

# runnable = ? | prompt | ....
# runnablbe.invoke(inputs)

## 5.1 用 RunnalbePassthrough.bind

In [1]:
from langchain.schema.runnable import RunnablePassthrough

add_by_pass = RunnablePassthrough.assign(e3=lambda x: x['e1'] + x['e2'])
print(add_by_pass.invoke({'e1': 2, 'e2': 3}))

{'e1': 2, 'e2': 3, 'e3': 5}


由此可見 `RunnablePassthrough.assign` 的功能可以理解成 `input[key] = create_value(input)`，在保有原資料下新增新的資料進去。

或是可以從字面上理解：把自己傳過去 (PassThrough) 同時外加指定一個值 (assign)

## 5.2 用 RunnalbeMap

另外一種等價的寫法

In [1]:
from operator import itemgetter
from langchain.schema.runnable import RunnableMap

add_by_map = RunnableMap({
    "e1": itemgetter('e1'), # 任意函式碰到 Runnable 會自動被包成 RunnableLambda 來串接
    "e2": itemgetter('e2'),
    "e3": lambda x: x['e1'] + x['e2']
})
add_by_map.invoke({'e1': 2, 'e2': 3})

{'e1': 2, 'e2': 3, 'e3': 5}

**想想看**

兩種寫法之間的優劣在哪？

<details>
<summary>參考</summary>
RunnableMap 的寫法個人覺得比較清楚。除非傳過來的資料有非常多的 key ，但我們只需要增加少量的 key，此時用 RunnablePassthrough.assign 可以避免把程式碼寫得太長。
</details>

# 6. Runtime Arguments
從之前的介紹我們可以發現有些重要的參數 (arguments) 是在呼叫模型 (model) 進行生成 (generation) 而非建立模型的物件 (initialization) 時傳入的，例如 `stop` 又或者是 `functions` 和 `function_call` 等等。在使用 LCEL (Langchain Expression Language) 時，我們可以透過 `bind` 來傳入這些參數 (arguments)

In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_setup import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([("human", "請告訴我用{material}製造出硫酸的步驟。")])
model = ChatOpenAI()

chain1 = prompt | model
output1 = chain1.invoke({'material': '白金'})
print(output1.content)
print("===========================================================================================")

chain2 = prompt | model.bind(stop=["2."])
output2 = chain2.invoke({'material': '白金'})
print(output2.content)

製造硫酸的主要步驟如下：

1. 原料準備：準備硫磺（硫黃）作為主要原料。硫磺可以從天然硫磺礦石中提取或經過硫磺氧化反應得到。

2. 硫磺氧化：將硫磺進行氧化反應，使其轉化為二氧化硫（SO2）氣體。這一步通常使用空氣或氧氣進行。

3. 催化轉化：將SO2氣體與空氣中的氧氣進行催化反應，產生硫三氧化（SO3）。

4. 吸收硫三氧化：將SO3氣體通過吸收劑，通常使用濕式吸收法，將SO3吸收到硫酸中。

5. 濃縮硫酸：將吸收到硫酸中的SO3與水反應，生成硫酸（H2SO4）。這一步通常需要將反應混合物進行冷卻和濃縮，以獲得高濃度的硫酸。

6. 精煉硫酸：將獲得的硫酸進行精煉和純化，以去除其中的雜質和不純物。

以上是一般用白金製造硫酸的步驟，然而，由於白金價格昂貴，實際製造硫酸時通常會使用其他相對便宜的催化劑，例如二氧化鉬或五氧化二鉬等。
白金製造硫酸的步驟如下：

1. 準備白金觸媒：將白金催化劑放入反應器中。白金通常以絲狀或顆粒狀存在，較常見的白金觸媒是以白金黑或白金網製成。


